# Browser Sandbox Documentation

## Description

Browser Sandbox is a secure browser execution environment that provides AI Agents with complete browser automation capabilities. The sandbox has a built-in browser service, supports visual operations via VNC, and programmatic control via CDP protocol.

### Key Features

- **Visual Interface**: Full desktop environment via VNC for direct browser operation
- **Programmatic Control**: Supports CDP for automated browser operations
- **File System Integration**: Seamless integration with sandbox file system, supporting file upload and download (see file system operations in e2b_base.ipynb)
- **Security Isolation**: Runs in isolated environment to ensure security

### Use Cases

- Web automation requiring login state
- Data scraping and crawling
- Saving browser operation results to sandbox file system
- Automation tasks requiring visual debugging


## Install Dependencies

In [ ]:
%pip install e2b playwright

## Create Browser Sandbox

In [ ]:
import os
from playwright.async_api import async_playwright
from e2b import Sandbox
from IPython.display import IFrame

# Get API Key and available domain from Agent Sandbox console
# Can be set via environment variables or modified directly here
if not os.getenv("E2B_DOMAIN"):
    os.environ["E2B_DOMAIN"] = "tencentags.com"
if not os.getenv("E2B_API_KEY"):
    os.environ["E2B_API_KEY"] = "your_api_key"

# Before using sandbox service, create a sandbox tool in the console, e.g., "browser-v1"
# Create browser sandbox, default runtime is 5 minutes, use timeout parameter to specify runtime, here we keep it running for 3600s (1h)
sandbox = Sandbox.create(template="browser-v1", timeout=3600)
print(f"browser sandbox created: {sandbox.sandbox_id}")

## VNC Visual Interface
Through VNC you can directly operate the browser in the sandbox, and subsequent browser behavior can also be observed via VNC


In [ ]:
# Build VNC access URL, you can directly operate the sandbox desktop in the browser
novnc_url = f"https://{sandbox.get_host(9000)}/novnc/vnc_lite.html?&path=websockify?access_token={sandbox._envd_access_token}"
print(f"vnc url: {novnc_url}")

IFrame(src=novnc_url, width="1920", height="1080")


## Basic Browser Control


In [ ]:
# Build CDP connection URL
cdp_url = f"https://{sandbox.get_host(9000)}/cdp"

async with async_playwright() as playwright:
    # Connect to remote browser via CDP protocol
    browser = await playwright.chromium.connect_over_cdp(
        cdp_url, 
        headers={"X-Access-Token": str(sandbox._envd_access_token)}
    )
    context = browser.contexts[0]
    page = context.pages[0]
    
    # Navigate to specified page
    await page.goto("https://tencent.com")
    await page.wait_for_load_state("networkidle")


## File Upload and Browser Integration

Upload local HTML file to sandbox, then open it in browser


In [ ]:
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Agent Sandbox Demo</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            padding: 50px;
            background-color: #f0f8ff;
        }
        h1 {
            color: #333;
            font-size: 3em;
        }
    </style>
</head>
<body>
    <h1>Welcome to Agent Sandbox</h1>
</body>
</html>
"""

with open("demo.html", "w", encoding="utf-8") as f:
    f.write(html_content)

with open("demo.html", "r", encoding="utf-8") as f:
    sandbox.files.write("demo.html", f)

async with async_playwright() as playwright:
    browser = await playwright.chromium.connect_over_cdp(
        cdp_url, 
        headers={"X-Access-Token": str(sandbox._envd_access_token)}
    )
    context = browser.contexts[0]
    page = context.pages[0]
    
    # Open local HTML file
    file_url = "file:///home/user/demo.html"
    await page.goto(file_url)
    await page.wait_for_load_state("networkidle")
    
    print({await page.title()})


In [ ]:
# Close sandbox
sandbox.kill()